In [122]:
import pandas as pd

In [124]:
df = pd.read_csv("incidentes_viales (2).csv", sep = ";",  low_memory = False , encoding = 'utf_8',encoding_errors = 'ignore')


In [125]:
df.head()

,AÑO,CBML,CLASE_ACCIDENTE,DIRECCION,DIRECCION ENCASILLADA,DISEÑO,EXPEDIENTE,FECHA_ACCIDENTE,FECHA_ACCIDENTES,MES,NRO_RADICADO,NUMCOMUNA,BARRIO,COMUNA,LOCATION,X,Y
0,2014,1105,Otro,CR 66 B CL 34,CR 066 B 034 000 00000,Tramo de via,A000095088,05/08/2014 07:15:00,2014-08-05T12:15:00.000Z,Con heridos,8,1451888,11,Los Conquistadores,Laureles Estadio,"[-75.589313202, 6.2397866157]",832677.92
1,2014,0509,Choque,CR 71 CL 103 D,CR 071 103 D 000 00000,Interseccion,A000095143,05/08/2014 14:30:00,2014-08-05T19:30:00.000Z,Con heridos,8,1451891,05,Girardot,Castilla,"[-75.5708179064, 6.3000411778]",834744.36
2,2014,1421,Choque,CR 49 CL 7,CR 049 007 000 00000,Tramo de via,A000095226,05/08/2014 15:50:00,2014-08-05T20:50:00.000Z,Con heridos,8,1451935,14,Patio Bonito,El Poblado,"[-75.5765743781, 6.20874755798]",834078.39
3,2014,0801,Otro,CR 39 A CL 66 C,CR 039 A 066 C 000 00000,Tramo de via,A000095227,05/08/2014 13:00:00,2014-08-05T18:00:00.000Z,Con heridos,8,1451956,08,Villa Hermosa,Villa Hermosa,"[-75.5512546684, 6.25905761463]",836897.02
4,2014,1017,Choque,CL 58 CR 37,CL 058 037 000 00000,Tramo de via,A000094940,05/08/2014 19:30:00,2014-08-06T00:30:00.000Z,Con heridos,8,1452022,10,Los Ángeles,La Candelaria,"[-75.5561495119, 6.24996631944]",836352.35


In [126]:
df.rename({'MES': 'GRAVEDAD','NRO_RADICADO':'MES','NUMCOMUNA':'NRO_RADICADO','BARRIO':'NUMCOMUNA','COMUNA':'BARRIO','LOCATION':'COMUNA','X': 'LOCATION','Y':'X'}, axis = 1, inplace = True)# Se reorganizan los nombres de la columna pues hay un
# problema en la descarga de los datos.
# OJO SOLO CORRER UNA VEZ

In [127]:
df.isna().sum()

AÑO                          0
CBML                     18156
CLASE_ACCIDENTE              6
DIRECCION                    0
DIRECCION ENCASILLADA      391
DISEÑO                    1148
EXPEDIENTE                 110
FECHA_ACCIDENTE              0
FECHA_ACCIDENTES             0
GRAVEDAD                     0
MES                          0
NRO_RADICADO                 5
NUMCOMUNA                    0
BARRIO                   19006
COMUNA                   12798
LOCATION                     0
X                            0
dtype: int64

## Eliminación de Variables

Se eliminan las siguientes variables. Expediente = Es un registro aleatorio sin análiss, NRo _ Radicao = Numero aleatorio, Direccion Encasillada = Se repite, FECHA_ACCIDENTES = Se repite, COMUNA = se tumba comuna pues tiene mucho nan y existe otra columna con la misma información , CBML = Mucha inconsistencia en caso de requerir el codigo catastral se puede buscar una página externa que reciba el código  catastraly entregue la localización , X = Debido a que existe un problema en la descarga de la base de datos no existe datos de la variable 'y' entonces es necesario eliminar la componente 'x' pues en ausencia de 'y', 'x' pierde sentido

In [128]:

df.drop(['EXPEDIENTE','NRO_RADICADO','DIRECCION ENCASILLADA', 'FECHA_ACCIDENTES','COMUNA','CBML','X'],axis = 1 , inplace = True)

In [129]:
df.isna().sum()

AÑO                    0
CLASE_ACCIDENTE        6
DIRECCION              0
DISEÑO              1148
FECHA_ACCIDENTE        0
GRAVEDAD               0
MES                    0
NUMCOMUNA              0
BARRIO             19006
LOCATION               0
dtype: int64

Debido a que existe un problema en la descarga de la base de datos no existe datos de la variable 'y' entonces es necesario eliminar la componente 'x' pues en ausencia de 'y', 'x' pierde sentido

In [130]:
df.shape

(270765, 10)

In [131]:
df.isna().sum()

AÑO                    0
CLASE_ACCIDENTE        6
DIRECCION              0
DISEÑO              1148
FECHA_ACCIDENTE        0
GRAVEDAD               0
MES                    0
NUMCOMUNA              0
BARRIO             19006
LOCATION               0
dtype: int64

In [132]:
df.columns

Index(['AÑO', 'CLASE_ACCIDENTE', 'DIRECCION', 'DISEÑO', 'FECHA_ACCIDENTE',
       'GRAVEDAD', 'MES', 'NUMCOMUNA', 'BARRIO', 'LOCATION'],
      dtype='object')

In [133]:
df['GRAVEDAD'].unique()

array(['Con heridos', 'Solo daños', 'Con muertos', 'Solo da\\xF1os'],
      dtype=object)

In [134]:
df['AÑO']= df['AÑO'].apply(lambda x : '2019' if x == '2019\\r' else x)
df['DISEÑO'] = df['DISEÑO'].apply(lambda x : 'Pontón' if x == 'Pont\\xF3n' else x)
df['HORA'] = df['FECHA_ACCIDENTE'].apply(lambda x: x.split(' ')[1])
df['FECHA_ACCIDENTE'] = df['FECHA_ACCIDENTE'].apply(lambda x: x.split(' ')[0])
df['GRAVEDAD'] = df['GRAVEDAD'].apply(lambda x: 'Solo daños' if x == 'Solo da\\xF1os' else x)

Para el número de comuna hay muchos datos que no son consistentes ('0','Sin Inf','In','SN','AU'). Se plantea la posibilidad de colocar 99 que corresponde a toda la ciudad es decir en cualquier parte o que no hay información. Se espera que con la localización se puede precisar

In [135]:
len(df[df['NUMCOMUNA'].isin(['0','Sin Inf','In','SN','AU'])])

20127

In [136]:
df['NUMCOMUNA'] = df['NUMCOMUNA'].apply(lambda x: '99' if x in(['0','Sin Inf','In','SN','AU']) else x)

In [137]:
def encondebarrio(x):
    
    z = {'Bel\\xE9n' : 'Belén', 'Belalc\\xE1zar' : 'Belalcázar','H\\xE9ctor Abad G\\xF3mez': 'Héctor Abad Gómez',
        'San Joaqu\\xEDn' : 'San Joaquín', 'Santa F\\xE9' : 'Santa Fé', 'Coraz\\xF3n de Jes\\xFAs' : 'Corazón de Jesús',
        'Alfonso L\\xF3pez': 'Alfonso López', 'Santa Mar\\xEDa de Los \\xC1ngeles' : 'Santa María de Los Ángeles',
        'El Rinc\\xF3n' : 'El Rincón', 'Alejandr\\xEDa': 'Alejandría', 'Campo Vald\\xE9s No. 1' : 'Campo Valdés No. 1',
        'Barrio Col\\xF3n' : 'Barrio Colón','Jard\\xEDn Bot\\xE1nico' : 'Jardín Botánico', 'Estaci\\xF3n Villa' : 'Estación Villa',
       'Barrio de Jes\\xFAs' : 'Barrio de Jesús', 'Barrio Crist\\xF3bal': 'Barrio Cristóbal', 'Boyac\\xE1' : 'Boyacá',
       'Mosc\\xFA No. 1':'Moscú No. 1', 'Sim\\xF3n Bol\\xEDvar' : 'Simón Bolívar', 'Nueva Villa de la Iguan\\xE1' : 'Nueva Villa de la Iguaná',
        'Bombon\\xE1 No. 2': 'Bomboná No. 2', 'La Pi\\xF1uela' : 'La Piñuela' , 'Play\\xF3n de Los Comuneros': 'Playón de Los Comuneros',
       'Jes\\xFAs Nazareno' : 'Jesús Nazareno', '\\xC1rea de Expansi\\xF3n Pajarito': 'Área de Expansión Pajarito', 'F\\xE1tima':'Fátima',
        'San Germ\\xE1n' : 'San Germán', 'Diego Echavarr\\xEDa':'Diego Echavarría', 'Mosc\\xFA No. 2':'Moscú No. 2', 'La Am\\xE9rica': 'La América',
        'Alejandro Echavarr\\xEDa':'Alejandro Echavarría','L\\xF3pez de Mesa':'López de Mesa', 'Los \\xC1ngeles' : 'Los Ángeles',
       '\\xC1rea de Expansi\\xF3n Altos de Calasanz':'Área de Expansión Altos de Calasanz' ,'Santa Mar\\xEDa de los \\xC1ngeles' : 'Santa María de los Ángeles', 
        'Santa M\\xF3nica': 'Santa Mónica', 'La Mansi\\xF3n': 'La Mansión', 'San Mart\\xEDn de Porres':'San Martín de Porres', 'C\\xF3rdoba': 'Córdoba',
       'Santa Luc\\xEDa': 'Santa Lucía', 'Batall\\xF3n Girardot': 'Batallón Girardot', 'El Vel\\xF3dromo':'El Velódromo', 'Andaluc\\xEDa': 'Andalucía', 
        'Bombon\\xE1 No.1' :'Bomboná No.1' , 'Mar\\xEDa Cano Carambolas': 'María Cano Carambolas', 'Nueva Villa de Aburr\\xE1' : 'Nueva Villa de Aburrá',
       '\\xC1rea de Expansi\\xF3n San Antonio de Prado': 'Área de Expansión San Antonio de Prado','Los Alc\\xE1zares' : 'Los Alcázares', 'El Coraz\\xF3n': 'El Corazón',
        'Catalu\\xF1a' : 'Cataluña', 'Berl\\xEDn': 'Berlín', 'Antonio Nari\\xF1o' : 'Antonio Nariño', 'Hospital San Vicente de Pa\\xFAl': 'Hospital San Vicente de Paúl',
       'San Jos\\xE9 de la Monta\\xF1a' : 'San José de la Montaña', 'Campo Vald\\xE9s No.2' : 'Campo Valdés No.2', 'San Jos\\xE9 la Cima No.2': 'San José la Cima No.2',
        'Mosc\\xFA No.2' : 'Moscú No. 2', 'San Jos\\xE9 la Cima No. 1': 'San José la Cima No. 1', 'Bombon\\xE1 No. 1' : 'Bomboná No. 1', 'Santa In\\xE9s': 'Santa Inés',
        'Campo Vald\\xE9s No. 2' : 'Campo Valdés No. 2'}
    
    
    if x in z:
        x = z[x]
    elif type(x) == float :
        x = None
    elif x[0] in ['0','1','2','3','4','5','6','7','8','9']:
        x = None
    else:
        x = x
    return x

In [138]:
df.isna().sum()

AÑO                    0
CLASE_ACCIDENTE        6
DIRECCION              0
DISEÑO              1148
FECHA_ACCIDENTE        0
GRAVEDAD               0
MES                    0
NUMCOMUNA              0
BARRIO             19006
LOCATION               0
HORA                   0
dtype: int64

In [139]:
df['BARRIO'] = df['BARRIO'].apply(encondebarrio)

In [141]:
df.isna().sum()

AÑO                    0
CLASE_ACCIDENTE        6
DIRECCION              0
DISEÑO              1148
FECHA_ACCIDENTE        0
GRAVEDAD               0
MES                    0
NUMCOMUNA              0
BARRIO             20613
LOCATION               0
HORA                   0
dtype: int64

Con las siguientes agrupaciones se puede inferir si es que tanta información se pierde. Podemos hacer como una distribución de los nan y los que no son nan

In [142]:
df_datos = df.groupby(['AÑO','MES'])[['BARRIO']].agg([ lambda x : x.isnull().sum(), lambda x : x.notnull().sum()])
df_datos.rename({df_datos.columns[0][1]:'Data perdida',df_datos.columns[1][1] : 'Datos completos'}, axis = 1, inplace = True)
df_datos.head(20)

BARRIO                
         Data perdida Datos completos
AÑO  MES                             
2014 7            231            3189
     8            307            3633
     9            290            3587
     10           266            3470
     11           275            3337
     12           292            3538
2015 1            283            2980
     2            226            3355
     3            329            3667
     4            274            3472
     5            348            3706
     6            287            3312
     7            290            3771
     8            329            3775
     9            294            3841
     10           268            3708
     11           290            3434
     12           279            3654
2016 1            241            3081
     2            289            3569

In [147]:
df.head()

,AÑO,CLASE_ACCIDENTE,DIRECCION,DISEÑO,FECHA_ACCIDENTE,GRAVEDAD,MES,NUMCOMUNA,BARRIO,LOCATION,HORA
0,2014,Otro,CR 66 B CL 34,Tramo de via,05/08/2014,Con heridos,8,11,Los Conquistadores,"[-75.589313202, 6.2397866157]",07:15:00
1,2014,Choque,CR 71 CL 103 D,Interseccion,05/08/2014,Con heridos,8,05,Girardot,"[-75.5708179064, 6.3000411778]",14:30:00
2,2014,Choque,CR 49 CL 7,Tramo de via,05/08/2014,Con heridos,8,14,Patio Bonito,"[-75.5765743781, 6.20874755798]",15:50:00
3,2014,Otro,CR 39 A CL 66 C,Tramo de via,05/08/2014,Con heridos,8,08,Villa Hermosa,"[-75.5512546684, 6.25905761463]",13:00:00
4,2014,Choque,CL 58 CR 37,Tramo de via,05/08/2014,Con heridos,8,10,Los Ángeles,"[-75.5561495119, 6.24996631944]",19:30:00
